<a href="https://colab.research.google.com/github/MaayanHagbi/CAS_ADS_Final_Project/blob/main/Fraud_Detection_ML_(CAS_ADS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Authenticate Google Big Query Access

In [12]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


##Connect to Big Query and Get Data

In [14]:
from google.cloud import bigquery
project_id = 'total-biplane-387513'
bigquery_client = bigquery.Client(project = project_id)


Query = "SELECT * FROM `total-biplane-387513.Data_Sceince_Project_Dataset.CAS_ADS_Dataset`;"
Query_Results = bigquery_client.query(Query)
df = Query_Results.to_dataframe()
df

,id,transaction_sn,cluster_sn,amount,amount_gbp,ip,processor_timestamp,arn,recurring_sn,dispute_age,...,MCCs_created_at,Accountmanager_id,Accountmanager_name,Accountmanager_updated_at,Accountmanager_created_at,Chargebackreason_id,Chargebackreason_code,Chargebackreason_description,Chargebackreason_updated_at,Chargebackreason_created_at
0,7095817,VqrXC2Gh,T2OuQf5h,640.000000000,506.340000000,91.230.225.25,2023-01-10 13:28:43+00:00,74405243010301123804611,None,<NA>,...,NaT,23,Hai Shmool,2020-04-18 23:08:37+00:00,NaT,<NA>,None,None,NaT,NaT
1,8387241,N8SZZ37U,yHHB67yr,53.270000000,53.270000000,51.186.147.125,2023-05-27 18:08:56+00:00,74836953149358891370883,None,<NA>,...,NaT,643,Nakisha Syed-Lawton,2024-03-27 13:16:02+00:00,NaT,<NA>,None,None,NaT,NaT
2,11008629,Cq3XjO8u,J6ck1cBQ,75.000000000,64.380000000,5.90.74.15,2024-02-28 07:40:36+00:00,75284534060358850021359,None,<NA>,...,NaT,643,Nakisha Syed-Lawton,2024-03-27 13:16:02+00:00,NaT,<NA>,None,None,NaT,NaT
3,7592987,0aeCSstH,nUkPeHIC,67.000000000,53.010000000,2603:8080:7940:282:cc51:9cdf:b62b:bac5,2023-03-04 16:14:50+00:00,74595343064000007644431,None,<NA>,...,NaT,28,Ori Levi,2020-04-18 23:08:37+00:00,NaT,<NA>,None,None,NaT,NaT
4,7896335,S6GgUEa5,EKgKvWeQ,67.000000000,53.010000000,2601:800:c102:a2f0:7582:19e:bbfe:d8d1,2023-04-03 21:05:33+00:00,74595343095000005749318,None,<NA>,...,NaT,349,Gabi Behiri,2024-03-27 14:00:43+00:00,NaT,<NA>,None,None,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492529,9491509,v7nSguMz,P6yQi8KT,26.960000000,26.960000000,185.189.113.166,2023-09-13 14:18:05+00:00,74836953257358939346737,None,<NA>,...,NaT,427,Gareth De Klerk,2024-03-27 13:21:07+00:00,NaT,<NA>,None,None,NaT,NaT
1492530,8016674,NsCRtuUR,3h49nbC7,23.390000000,23.390000000,89.243.103.146,2023-04-16 19:01:07+00:00,75284533107358867192687,None,<NA>,...,NaT,427,Gareth De Klerk,2024-03-27 13:21:07+00:00,NaT,<NA>,None,None,NaT,NaT
1492531,9606930,2fSMgUdt,1w7TpuGc,25.650000000,25.650000000,82.132.219.199,2023-09-28 12:04:41+00:00,75284533272358850256175,None,<NA>,...,NaT,427,Gareth De Klerk,2024-03-27 13:21:07+00:00,NaT,<NA>,None,None,NaT,NaT
1492532,10803944,ja12Wz6i,kdziRbIo,18.430000000,18.430000000,90.248.18.114,2024-02-04 21:18:32+00:00,75284534036358836019411,None,<NA>,...,NaT,427,Gareth De Klerk,2024-03-27 13:21:07+00:00,NaT,<NA>,None,None,NaT,NaT
